In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
# 학습 데이터(MNIST)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot=True)

In [ ]:
# 이미지와 라벨의 입력
X = tf.placeholder(tf.float32, [None, 28*28]) # 무한대 x 784(28*28) 행렬
Y = tf.placeholder(tf.int64, [None]) # 무한대 x 1 행렬

In [ ]:
batch_size=100
learning_rate=0.01

In [ ]:
# MLP Model
def model(input_X):
    # 히든 레이어 1
    W1 = tf.Variable(tf.truncated_normal([28*28, 100], stddev=0.1))
    b1 = tf.Variable(tf.zeros([batch_size]))
    h1 = tf.nn.sigmoid(tf.matmul(input_X, W1) + b1)
#     h1 = tf.nn.tanh(tf.matmul(input_X, W1) + b1)
    
    
#     # 히든 레이어2
#     W2 = tf.Variable(tf.truncated_normal([100, 50], stddev=0.1))
#     b2 = tf.Variable(tf.zeros([50]))
#     h2 = tf.nn.tanh(tf.matmul(h1, W2) + b2)

    # 출력(fully connected) 레이어 (10개 출력)
    class_num = 10
    W_fc = tf.Variable(tf.truncated_normal([100, class_num], stddev=0.1))
    b_fc = tf.Variable(tf.zeros([class_num]))
    pred = tf.matmul(h1, W_fc) + b_fc
#     pred = tf.matmul(h2, W_fc) + b_fc

    return pred

In [ ]:
pred = model(X)

In [ ]:
# Cost Function 설계 (Cross Entropy)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=pred)
cost = tf.reduce_mean(cross_entropy)

# Gradient descent Optimizer(학습)
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [ ]:
# 학습 시작
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(10000):
    X_images, X_labels = mnist.train.next_batch(batch_size)
    X_labels = np.where(X_labels)[1]
#     print(X_labels)
        
    sess.run(optimizer, feed_dict={X: X_images, Y: X_labels})

    # 로그
    training_cost = sess.run(cost, feed_dict={X: X_images, Y: X_labels})
    print(epoch, training_cost)

print("학습완료! (cost : " + str(training_cost) + ")")

In [ ]:
# test_images, test_labels = mnist.test.next_batch(16)
# test_labels = np.where(test_labels)[1]
    
test_input = 100
nCnt = 0
for i in range(1):
    x_test, test_label = mnist.test.next_batch(test_input)
    arr_data = sess.run(pred, feed_dict={X: x_test})

    pred_val = tf.argmax(arr_data, 1)
    real_val = np.where(test_label)[1]
    
    for ib in range(test_input):
        tmp_pred = pred_val.eval(session=sess)
        tmp_real = real_val
        
        if tmp_pred[ib] == tmp_real[ib]:
            print("예측값: " + str(tmp_pred[ib]) + " / 실제값: " + str(tmp_real[ib]) + " => True")
            nCnt += 1
        else:
            print("예측값: " + str(tmp_pred[ib]) + " / 실제값: " + str(tmp_real[ib]) + " => False")

print("Accuracy:", (nCnt/100.0)*100)